In [ ]:
import os
from dotenv import load_dotenv

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

: 

In [ ]:
load_dotenv()

In [ ]:
api_key = os.getenv("GOOGLE_API_KEY") # GEMINI_API_KEY 키에 해당하는 value를 가져온다 (=api key)
print(api_key[:-5]) # 보안상의 이유로 끝의 5자리는 생략!|

In [ ]:
# api key를 등록, 저장
# 이 코드를 실행함으로써 Gemini 모델 객체를 만들거나 generate_content()와 같은 api 호출 메소드를 반복해서 실행할 때 매번 key를 넘기지 않아도 된다. -> 일종의 프로젝트 key 전역설정
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Gemini API 가격 및 사용 가능 토큰 확인
https://ai.google.dev/gemini-api/docs/pricing?hl=ko

### 현재 무료로 사용할 수 있는 Gemini 모델 종류

1. Gemini 2.5 Flash
2. Gemini 2.5 Flash-Lite
3. Gemini 2.5 Pro
4. Gemini Embeddings 모델


|모델|RPM (분당 요청)|TPM (분당 토큰)|RPD (일일 요청)|
|--|--|--|--|
|Gemini 2.5 Pro|5|250,000|100|
|Gemini 2.5 Flash|10|250,000|250|
|Gemini 2.5 Flash-Lite|15|250,000|1,500|

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [ ]:
model

# Chapter 1. Gemini에 직접 질문을 해서 답변을 받아보자!

## reponse : LLM 모델의 답변
- response.text → 최종 생성 텍스트
- response.candidates → 여러 후보 응답
- response.usage_metadata → 토큰 사용량 정보

etc...

In [ ]:
# 1. invoke(text) 내의 문장(text)을 프롬프트로 LLM 모델에게 전달
# 2. invoke() : Google Gemini API 서버에 요청
# 3. response : LLM 모델이 내놓은 답변 (여러 정보 확인 가능)
response = model.invoke("안녕하세요, Gemini API 테스트 중입니다.") # 질문 입력

In [ ]:
# 실제 답변을 확인
print(response.content)

# Chapter 2. 우리의 대화를 기억하게 하자
- 지금까지의 대화 세션을 저장, 메모리 캐싱해서 추가적인 질문 없이 Gemini가 직접 대화를 검색하도록 하는 방법
> 동일한 질문에 대한 답변을 저장, 메모리에 저장된 답변을 반환
- 토큰 사용량이 줄어듦
- 대답 속도 향상

In [ ]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

In [ ]:
%%time

# Gemini 모델 불러오기
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Gemini에 질문 입력하기 위한 템플릿 생성
prompt = PromptTemplate.from_template("{country}에 대해서 설명해줘. 최대한 짧게!")

# 프롬프트 생성부터 모델 불러오기, 답변 출력까지 하나의 체인으로 정의 -> 'chain'이라는 이름으로 만들어준다
chain = prompt | llm | StrOutputParser()

print('Gemini의 답변 : ', chain.invoke({"country": "한국"}))

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response)

In [ ]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "미국"})
print(response)

# Chapter 3. 우리가 원하는 포맷으로 답변을 받아보자!
- Zero Shot 프롬프트 : 원하는 답변 포맷을 제공하지 않음
- One Shot 프롬프트 : 원하는 답변 포맷을 한 가지 예시로 제공하는 방법
- Few Shot 프롬프트 : 원하는 답변 포맷을 여러 예시로 제공하는 방법

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate

In [ ]:
examples = [
    {
        "question": "최근 가장 급등한 주식 종목 알려줘",

        "answer": """
추가 질문: 대표적인 미국 주식 중 3개를 알려줘
중간 답변: 1. 엔비디아 2. 구글 3.메타
최종 답변은: 엔비디아, 테슬라, 팔란티어가 가장 많이 올랐습니다.
"""},
    {
        "question": "최근 가장 급등한 코인 알려줘",

        "answer": """
추가 질문: 대표적인 코인 3개를 알려줘
중간 답변: 1. 비트코인 2. 이더리움 3. 도지
최종 답변은: 비트코인, 이더리움, 리플이 가장 많이 올랐습니다.
"""
    }
]

In [ ]:
example_prompt = PromptTemplate.from_template(
"Question:\n{question}\nAnswer:\n{answer}"
)

print(example_prompt.format(**examples[0]))

##### promt : 실제 LLM 모델이 받아드리는 질문 포맷 (학습 완료!)

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

#### 첫번째, 두번째 Question, Answer : 우리가 원하는 질문에 따른 답변의 포맷
#### 마지막 Question : 실제 질문

In [ ]:
# 새로운 질문
question = "최근 1년 동안 가장 많이 오른 한국 주식 알려줘"
final_prompt = prompt.format(question=question)
print(final_prompt)

In [ ]:
response = model.invoke(final_prompt)

In [ ]:
print(response.content)

# 주의사항
- 예시 포맷도 토큰 비용에 포함되기 때문에 토큰 사용량을 고려한 적절한 처리가 필요